{{ badge }}



```
function ClickConnect(){
    console.log("Clicked on connect button"); 
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,60000)
```





```
d=[]
while(1):
  d.append('1')
```



In [0]:
!pip install fastparquet

     |████████████████████████████████| 153kB 2.7MB/s 
     |████████████████████████████████| 61kB 4.7MB/s 
  Created wheel for fastparquet: filename=fastparquet-0.3.3-cp36-cp36m-linux_x86_64.whl size=253322 sha256=658c398455adced37e109fa49ce64b1b8df3a86331e59023f29d6b2990a3a0f5
  Stored in directory: /root/.cache/pip/wheels/a0/27/9f/d8066bbbbb77e97d8ad3daf4de155ead73693bc4aa2f52098c
  Created wheel for thrift: filename=thrift-0.13.0-cp36-cp36m-linux_x86_64.whl size=345240 sha256=ae07d35cf001755497987252d66fd50c18ef18368a97067545cb52cdc006e6bf
  Stored in directory: /root/.cache/pip/wheels/02/a2/46/689ccfcf40155c23edc7cdbd9de488611c8fdf49ff34b1706e
Successfully built fastparquet thrift


In [0]:
!pip install pyspark
!pip install hyperopt --upgrade

     |████████████████████████████████| 217.8MB 44kB/s 
     |████████████████████████████████| 204kB 50.3MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=f20dabf4356ddde17b9e29b8397ca937444dcd2d7aa6c1e12c125c178f5290a6
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
     |████████████████████████████████| 972kB 2.8MB/s 
  Found existing installation: hyperopt 0.1.2
    Uninstalling hyperopt-0.1.2:
      Successfully uninstalled hyperopt-0.1.2


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import csv
import datetime as dt
import math
import warnings
import gc
import random
import pickle
import pyspark
import fastparquet

from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDRegressor
from mlxtend.classifier import StackingClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

#metrics
from sklearn.metrics import r2_score #r-square error
from sklearn.metrics import mean_squared_error #MSE, sqrt(MSE)= RMSE


from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.metrics import precision_recall_curve, auc, roc_curve
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import cross_val_score

from collections import Counter
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings('ignore')
from tqdm import tqdm,tnrange

from hyperopt import hp,fmin,tpe,STATUS_OK,space_eval,SparkTrials

#viz
import plotly.graph_objects as go

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.



In [0]:
price_data = pd.read_parquet('/content/drive/My Drive/Colab Notebooks/Works/Luggage bags/price_ft.parquet.gzip',engine='fastparquet')

##Splitting the data

In [0]:
price_data.head()

,Weight,Weight1,Length,Height,Width,Extra_Wt,Volume,Cost
0,23.2,25.4,30.0,11.5200,4.0200,2.2,1389.3120,242.0
1,24.0,26.3,31.2,12.4800,4.3056,2.3,1676.4973,290.0
2,23.9,26.5,31.1,12.3778,4.6961,2.6,1807.7617,340.0
3,26.3,29.0,33.5,12.7300,4.4555,2.7,1900.0703,363.0
4,26.5,29.0,34.0,12.4440,5.1340,2.5,2172.1749,430.0


In [0]:
price_data.shape

(158, 8)

In [0]:
price_data.head()

,Weight,Weight1,Length,Height,Width,Extra_Wt,Volume,Cost
0,23.2,25.4,30.0,11.5200,4.0200,2.2,1389.3120,242.0
1,24.0,26.3,31.2,12.4800,4.3056,2.3,1676.4973,290.0
2,23.9,26.5,31.1,12.3778,4.6961,2.6,1807.7617,340.0
3,26.3,29.0,33.5,12.7300,4.4555,2.7,1900.0703,363.0
4,26.5,29.0,34.0,12.4440,5.1340,2.5,2172.1749,430.0


In [0]:
#As each feature has different units. Lets standardize the data
std_scaler = StandardScaler()
price_data_ar = std_scaler.fit_transform(price_data)
price_data_sc = pd.DataFrame(data=price_data_ar, columns=price_data.columns)

In [0]:
price_data_sc.head()

,Weight,Weight1,Length,Height,Width,Extra_Wt,Volume,Cost
0,-0.309933,-0.286587,-0.110466,0.591654,-0.240092,0.034596,-0.211883,-0.445495
1,-0.229770,-0.202457,-0.006935,0.815870,-0.070461,0.161718,-0.023416,-0.310877
2,-0.239791,-0.183761,-0.015562,0.792000,0.161475,0.543083,0.062727,-0.170649
3,0.000698,0.049934,0.191499,0.874260,0.018572,0.670204,0.123305,-0.106145
4,0.020738,0.049934,0.234637,0.807462,0.421564,0.415961,0.301875,0.081760


In [0]:
price_data_sc.reset_index(inplace=True)
y_true = price_data_sc['Cost']
price_data_sc.drop(labels=['index','Cost'], axis=1, inplace = True)

In [0]:
price_data_sc.head()

,Weight,Weight1,Length,Height,Width,Extra_Wt,Volume
0,-0.309933,-0.286587,-0.110466,0.591654,-0.240092,0.034596,-0.211883
1,-0.229770,-0.202457,-0.006935,0.815870,-0.070461,0.161718,-0.023416
2,-0.239791,-0.183761,-0.015562,0.792000,0.161475,0.543083,0.062727
3,0.000698,0.049934,0.191499,0.874260,0.018572,0.670204,0.123305
4,0.020738,0.049934,0.234637,0.807462,0.421564,0.415961,0.301875


* Have to do k'-fold cross validation to find best hyper parameter
* First we'll go by using hyper opt on all algos
* Then we find using cross validation and K'-fold cross validation
* For each method, we evaluate model using all three metrics.
* Once we write functions for each then we can use them for all

In [0]:
price_data_sc.shape

(158, 7)

#Finding best parameter by training whole training data

In [0]:
#we are not using stratify here, as it's only for classed data to maintain the proportion
#of splitting same as the original data.
X_train, X_test, y_train, y_test = train_test_split(price_data_sc, y_true, test_size=0.2, random_state = 42)

print('X_train : ', X_train.shape)
print('y_train : ', y_train.shape)
print('X_test : ', X_test.shape)
print('y_test : ', y_test.shape)

X_train :  (126, 7)
y_train :  (126,)
X_test :  (32, 7)
y_test :  (32,)


* **Function for Linear regression**

In [0]:
def Lr_training_model(X_train, y_train, X_test, y_test):

  #training the model with found best C and testing on x_test data

  model_lr = LinearRegression()
  model_lr.fit(X_train,y_train)

  y_pred_train = model_lr.predict(X_train)
  y_pred_test = model_lr.predict(X_test)

  
  lr_r2_train = r2_score(y_train, y_pred_train)
  lr_r2_test = r2_score(y_test, y_pred_test)

  print(f'r2_score on train data is {lr_r2_train}')
  print(f'r2_score on test data is {lr_r2_test}')
  print(' ')

  lr_mse_train = mean_squared_error(y_train, y_pred_train)
  lr_mse_test = mean_squared_error(y_test, y_pred_test)

  print(f'MSE on train data is {lr_mse_train}')
  print(f'MSE on test data is {lr_mse_test}')
  print(' ')

  lr_rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
  lr_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

  print(f'rmse on train data is {lr_rmse_train}')
  print(f'rmse on test data is {lr_rmse_test}')
  print(' ')


In [0]:
Lr_training_model(X_train, y_train, X_test, y_test)

r2_score on train data is 0.9557413973929311
r2_score on test data is 0.979941107439705
 
MSE on train data is 0.04339443482701778
MSE on test data is 0.02117951845207843
 
rmse on train data is 0.20831330928919972
rmse on test data is 0.14553184686548312
 


#Finding best hyper parameter using CV but with only 1 fold

In [0]:
#we are not using stratify here, as it's only for classed data to maintain the proportion
#of splitting same as the original data.
X_train_tmp, X_test, y_train_tmp, y_test = train_test_split(price_data_sc, y_true, test_size=0.2, random_state = 42)

X_train, X_cv, y_train, y_cv = train_test_split(X_train_tmp, y_train_tmp, test_size=0.2)


print('X_train : ', X_train.shape)
print('y_train : ', y_train.shape)
print('X_cv : ', X_cv.shape)
print('y_cv : ', y_cv.shape)
print('X_test : ', X_test.shape)
print('y_test : ', y_test.shape)

X_train :  (100, 7)
y_train :  (100,)
X_cv :  (26, 7)
y_cv :  (26,)
X_test :  (32, 7)
y_test :  (32,)


## Functions for finding best hyper parameter

In [0]:
def err_plot(error_d,title,y_label):

  #plotting the errors vs C
  X_alpha, Y_err = zip(*error_d.items())

  f, ax = plt.subplots(figsize=(12,8))
  
  sns.lineplot(x=X_alpha, y=Y_err, markers=True)
  sns.scatterplot(x=X_alpha, y=Y_err)

  for i,txt in enumerate(Y_err):
    ax.text(X_alpha[i],txt, (X_alpha[i], txt), fontsize=12)
  
  plt.grid()
  plt.title(title)
  plt.xlabel('Alpha')
  plt.ylabel(y_label)
  plt.show()

In [0]:
def err_plot(error_d):
  
  fig = go.Figure()

  for i in error_d.columns[1:]:
    fig.add_trace(go.Scatter(x = error_d.iloc[:,0],
                             y = error_d[i],
                             name = i,
                             line = dict(width=3)))
  
  fig.update_layout(
      autosize=False,
      width=1100,
      height=650,
      title='Alphs Vs Metric',
      title_x = 0.5,
      xaxis_title = 'Alpha',
      yaxis_title = 'Computed Score/Error'
  )

  fig.show()

In [0]:
def train_test_plot(tr_ts_er):
  fig = go.Figure(data=[
                      go.Bar(x=tr_ts_er.iloc[:,0], y=tr_ts_er.iloc[:,1], name = tr_ts_er.columns[1]),
                      go.Bar(x=tr_ts_er.iloc[:,0], y=tr_ts_er.iloc[:,2], name = tr_ts_er.columns[2])
  ])

  fig.update_layout(barmode='group',
                    autosize=False,
                    width =800,
                    height=650,
                    title='Train Vs Test Error',
                    title_x = 0.5,
                    xaxis_title = 'Alpha',
                    yaxis_title = 'Computed Score/Error')
  fig.show()

In [0]:
def training_model(alpha_r2, alpha_mse, alpha_rmse, r2_error, mse, rmse, reg, loss, X_train, y_train, X_test, y_test):

  #training the model with found best C and testing on x_test data
  #alpha wrt r2
  model_sgd_r2 = SGDRegressor(alpha = alpha_r2, penalty=reg, loss = loss, random_state=42)
  model_sgd_r2.fit(X_train,y_train)
  y_pred_r2 = model_sgd_r2.predict(X_test)
  
  sgd_r2_test = r2_score(y_test, y_pred_r2)

  print(f'r2_score at alpha : {alpha_r2} on CV/train data is {r2_error[alpha_r2]}')
  print(f'r2_score at alpha : {alpha_r2} on test data is {sgd_r2_test}')
  print(' ')

  tr_ts_vals= list([list([str(alpha_r2)+'(r2)',r2_error[alpha_r2], sgd_r2_test])])

  #alpha wrt mse
  model_lgr_mse = SGDRegressor(alpha = alpha_mse, penalty=reg, loss = loss, random_state=42)
  model_lgr_mse.fit(X_train,y_train)
  y_pred_mse = model_lgr_mse.predict(X_test)
  
  sgd_mse_test = mean_squared_error(y_test, y_pred_mse)

  print(f'MSE at alpha : {alpha_mse} on CV/train data is {mse[alpha_mse]}')
  print(f'MSE at alpha : {alpha_mse} on test data is {sgd_mse_test}')
  print(' ')

  tr_ts_vals.append(list([str(alpha_mse)+'(MSE)',mse[alpha_mse], sgd_mse_test]))


  #alpha wrt rmse
  model_sgd_rmse = SGDRegressor(alpha = alpha_rmse, penalty=reg, loss = loss, random_state=42)
  model_sgd_rmse.fit(X_train,y_train)
  y_pred_rmse = model_sgd_rmse.predict(X_test)
  
  sgd_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_rmse))

  print(f'rmse at alpha : {alpha_rmse} on CV/train data is {rmse[alpha_rmse]}')
  print(f'rmse at alpha : {alpha_rmse} on test data is {sgd_rmse_test}')
  print(' ')

  tr_ts_vals.append(list([str(alpha_rmse)+'(RMSE)', rmse[alpha_rmse], sgd_rmse_test]))

  tr_ts_er_df = pd.DataFrame(data=tr_ts_vals, columns=['alpha', 'Train', 'Test'])

  train_test_plot(tr_ts_er_df)



In [0]:
def best_alpha(alpha_limit, reg, loss, X_train, y_train, X_cv, y_cv, X_test, y_test):
  
  r2_error = dict()
  mse = dict()
  rmse = dict()

  for i in alpha_limit:
    model_sgd = SGDRegressor(alpha = i, penalty = reg, loss = loss, random_state = 42)
    model_sgd.fit(X_train, y_train)

    #predicting using cv data
    y_pred = model_sgd.predict(X_cv)

    #getting r2_error
    r2_error[i] = r2_score(y_cv,y_pred)

    #getting MSE
    mse[i] = mean_squared_error(y_cv,y_pred)

    #getting RMSE
    rmse[i] = np.sqrt(mean_squared_error(y_cv,y_pred))
    #above all are the errors and error has to be minimum
    
    print(i, ' ', r2_error[i], ' ', mse[i], ' ', rmse[i])
  
  error_df =pd.DataFrame(data=r2_error.items(),columns=['Alpha','r2_score'])
  error_df['MSE'] = [v for k,v in mse.items()]
  error_df['RMSE'] = [v for k,v in rmse.items()]

  #finding best alpha wrt each error
  alpha_r2 = max(r2_error, key=r2_error.get)
  min_r2 = r2_error[alpha_r2]

  alpha_mse = min(mse,key=mse.get)
  min_mse = mse[alpha_mse]

  alpha_rmse = min(rmse, key=rmse.get)
  min_rmse = rmse[alpha_rmse]

  #plotting errors vs alpha
  #err_plot(r2_error,'r2 error Vs. Alpha', 'r2 error')
  #err_plot(mse,'MSE Vs. Alpha', 'MSE')
  #err_plot(rmse,'RMSE Vs. Alpha', 'RMSE')

  err_plot(error_df)

  
  #training the model with found best C and testing on x_test data
  print('Training the model with found best alpha')
  training_model(alpha_r2, alpha_mse, alpha_rmse, r2_error, mse, rmse, reg, loss, X_train, y_train, X_test, y_test)

  return r2_error, mse, rmse

##Making predictions

In [0]:
#Training SGD with l2 reg and squared loss
alpha_limit = [10**x for x in range(-5,2)]
reg = 'l2'
loss = 'squared_loss'
r2_error, mse, rmse = best_alpha(alpha_limit, reg, loss, X_train, y_train, X_cv, y_cv, X_test, y_test)

1e-05   0.9170031971018894   0.04892669032223418   0.22119378454702152
0.0001   0.9169997243852485   0.04892873749183267   0.2211984120463632
0.001   0.9169649899711427   0.04894921346034156   0.22124469137211306
0.01   0.9166169398914096   0.049154389296917254   0.22170789182371758
0.1   0.9130820640842866   0.05123820178009189   0.22635856904498203
1   0.8770604953285773   0.07247294911843512   0.26920800344424217
10   0.5355990205092349   0.2737647971426154   0.5232253789167871


Training the model with found best alpha
r2_score at alpha : 1e-05 on CV/train data is 0.9170031971018894
r2_score at alpha : 1e-05 on test data is 0.9497296110017946
 
MSE at alpha : 1e-05 on CV/train data is 0.04892669032223418
MSE at alpha : 1e-05 on test data is 0.053078834147012866
 
rmse at alpha : 1e-05 on CV/train data is 0.22119378454702152
rmse at alpha : 1e-05 on test data is 0.23038844186940644
 


In [0]:
alpha_limit = [10**x for x in range(-5,2)]
reg = 'l1'
loss = 'squared_loss'
r2_error, mse, rmse = best_alpha(alpha_limit, reg, loss, X_train, y_train, X_cv, y_cv, X_test, y_test)

1e-05   0.9170053150056273   0.04892544181606649   0.22119096232908453
0.0001   0.9170214415368284   0.04891593521134803   0.22116947169839699
0.001   0.917193383852035   0.048814575061129614   0.2209402069817298
0.01   0.917843039189915   0.04843160265228075   0.22007181248919805
0.1   0.8977823408431169   0.06025740245888856   0.24547383253391503
1   0.008934791133622277   0.5842338363668396   0.7643519061053223
10   -0.00015775894976122196   0.5895939028589819   0.7678501825610137


Training the model with found best alpha
r2_score at alpha : 0.01 on CV/train data is 0.917843039189915
r2_score at alpha : 0.01 on test data is 0.9503921945904823
 
MSE at alpha : 0.01 on CV/train data is 0.04843160265228075
MSE at alpha : 0.01 on test data is 0.052379234141654976
 
rmse at alpha : 0.01 on CV/train data is 0.22007181248919805
rmse at alpha : 0.01 on test data is 0.22886510031382018
 


#Fiding best hyper parameter using K'-fold CV

* So we took only 26 points for cv and 100 points for training the data out of total 158 points.
* As the data we have at hand is very less and we lost 26 points for CV, now we use K-fold CV to use those 26 points as well to train the data.

* All the functions remains same except best_alpha as we need to incorporate cv code.

## Functions for finding best hyper parameter using K'-fold CV using GridSearch CV

In [0]:
import plotly.graph_objects as go

In [0]:
#we are not using stratify here, as it's only for classed data to maintain the proportion
#of splitting same as the original data.
X_train, X_test, y_train, y_test = train_test_split(price_data_sc, y_true, test_size=0.2, random_state = 42)

print('X_train : ', X_train.shape)
print('y_train : ', y_train.shape)
print('X_test : ', X_test.shape)
print('y_test : ', y_test.shape)

X_train :  (126, 7)
y_train :  (126,)
X_test :  (32, 7)
y_test :  (32,)


In [0]:
def err_plot(error_d):
  
  fig = go.Figure()

  for i in error_d.columns[1:]:
    fig.add_trace(go.Scatter(x = error_d.iloc[:,0],
                             y = error_d[i],
                             name = i.split('_')[2],
                             line = dict(width=3)))
  
  fig.update_layout(
      autosize=False,
      width=1100,
      height=650,
      title='Alphs Vs Metric',
      title_x = 0.5,
      xaxis_title = 'Alpha',
      yaxis_title = 'Computed Score/Error'
  )

  fig.show()

In [0]:
def train_test_plot(tr_ts_er):
  fig = go.Figure(data=[
                      go.Bar(x=tr_ts_er.iloc[:,0], y=tr_ts_er.iloc[:,1], name = tr_ts_er.columns[1]),
                      go.Bar(x=tr_ts_er.iloc[:,0], y=tr_ts_er.iloc[:,2], name = tr_ts_er.columns[2])
  ])

  fig.update_layout(barmode='group',
                    autosize=False,
                    width =800,
                    height=650,
                    title='Train Vs Test Error',
                    title_x = 0.5,
                    xaxis_title = 'Alpha',
                    yaxis_title = 'Computed Score/Error')
  fig.show()

In [0]:
def training_model(alpha_r2, alpha_mse, alpha_rmse, r2_vals, mse, rmse, reg, loss, X_train, y_train, X_test, y_test):

  #training the model with found best C and testing on x_test data
  #alpha wrt r2
  model_sgd_r2 = SGDRegressor(alpha = alpha_r2, penalty=reg, loss = loss, random_state=42)
  model_sgd_r2.fit(X_train,y_train)
  y_pred_r2 = model_sgd_r2.predict(X_test)
  
  predic_r2 = X_test.copy()
  predic_r2['Pred'] = y_pred_r2

  sgd_r2_test = r2_score(y_test, y_pred_r2)

  print(f'r2_score at alpha : {alpha_r2} on CV/train data is {r2_vals[alpha_r2]}')
  print(f'r2_score at alpha : {alpha_r2} on test data is {sgd_r2_test}')
  print(' ')

  tr_ts_vals= list([list([str(alpha_r2)+'(r2)',r2_vals[alpha_r2], sgd_r2_test])])

  #alpha wrt mse
  model_lgr_mse = SGDRegressor(alpha = alpha_mse, penalty=reg, loss = loss, random_state=42)
  model_lgr_mse.fit(X_train,y_train)
  y_pred_mse = model_lgr_mse.predict(X_test)

  predic_mse = X_test.copy()
  predic_mse['Pred'] = y_pred_mse

  sgd_mse_test = mean_squared_error(y_test, y_pred_mse)

  print(f'MSE at alpha : {alpha_mse} on CV/train data is {mse[alpha_mse]}')
  print(f'MSE at alpha : {alpha_mse} on test data is {sgd_mse_test}')
  print(' ')

  tr_ts_vals.append(list([str(alpha_mse)+'(MSE)',mse[alpha_mse], sgd_mse_test]))

  #alpha wrt rmse
  model_sgd_rmse = SGDRegressor(alpha = alpha_rmse, penalty=reg, loss = loss, random_state=42)
  model_sgd_rmse.fit(X_train,y_train)
  y_pred_rmse = model_sgd_rmse.predict(X_test)
  
  predic_rmse = X_test.copy()
  predic_rmse['Pred'] = y_pred_rmse

  sgd_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_rmse))

  print(f'rmse at alpha : {alpha_rmse} on CV/train data is {rmse[alpha_rmse]}')
  print(f'rmse at alpha : {alpha_rmse} on test data is {sgd_rmse_test}')
  print(' ')

  tr_ts_vals.append(list([str(alpha_rmse)+'(RMSE)', rmse[alpha_rmse], sgd_rmse_test]))

  tr_ts_er_df = pd.DataFrame(data=tr_ts_vals, columns=['alpha', 'Train', 'Test'])

  train_test_plot(tr_ts_er_df)

  return predic_r2, predic_mse, predic_rmse

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error, make_scorer

def best_alpha_cv(alpha_limit, reg, loss, X_train, y_train, X_test, y_test):

  def sqrot(yt,yp): return np.sqrt(mean_squared_error(yt,yp))

  metrics ={'r2':make_scorer(r2_score), 
            'mse':make_scorer(mean_squared_error, greater_is_better=False),
            'rmse':make_scorer(sqrot, greater_is_better=False)}

  model_sgd = SGDRegressor(penalty = reg, loss = loss, random_state = 42)
  model_gds = GridSearchCV(estimator=model_sgd, param_grid = alpha_limit, cv=5, scoring=metrics, refit = False)
  model_gds.fit(X_train, y_train)

  error_df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in model_gds.cv_results_.items()]))

  r2_vals = dict(zip(error_df.param_alpha, error_df.mean_test_r2))
  mse = dict(zip(error_df.param_alpha, error_df.mean_test_mse))
  rmse = dict(zip(error_df.param_alpha, error_df.mean_test_rmse))

  #finding best alpha wrt each error
  alpha_r2 = max(r2_vals, key=r2_vals.get)
  min_r2 = r2_vals[alpha_r2]

  alpha_mse = min(mse,key=mse.get)
  min_mse = mse[alpha_mse]

  alpha_rmse = min(rmse, key=rmse.get)
  min_rmse = rmse[alpha_rmse]

  #plotting errors vs alpha
  err_plot(error_df[['param_alpha','mean_test_r2','mean_test_mse','mean_test_rmse']])


  #training the model with found best C and testing on x_test data
  print('Training the model with found best alpha')
  predic_r2, predic_mse, predic_rmse = training_model(alpha_r2, alpha_mse, alpha_rmse, r2_vals, mse, rmse, reg, loss, X_train, y_train, X_test, y_test)

  return predic_r2, predic_mse, predic_rmse

##Making predictions

In [0]:
#driver function
alpha_limit = {'alpha': [10**x for x in range(-2,5)]}
reg = 'l1'
loss = 'squared_loss'
predic_r2, predic_mse, predic_rmse = best_alpha_cv(alpha_limit, reg, loss, X_train, y_train, X_test, y_test)

Training the model with found best alpha
r2_score at alpha : 0.01 on CV/train data is 0.9015681366513741
r2_score at alpha : 0.01 on test data is 0.9530552044133485
 
MSE at alpha : 10 on CV/train data is -0.9907503142179476
MSE at alpha : 10 on test data is 1.0809749118104697
 
rmse at alpha : 10 on CV/train data is -0.9818331781476026
rmse at alpha : 10 on test data is 1.039699433399129
 


* Scores and errors got better when CV is used and for the same range of alpha's.

#Finding best hyper parameter using Random Search CV

In [0]:
import plotly.graph_objects as go

In [0]:
#we are not using stratify here, as it's only for classed data to maintain the proportion
#of splitting same as the original data.
X_train, X_test, y_train, y_test = train_test_split(price_data_sc, y_true, test_size=0.2, random_state = 42)

print('X_train : ', X_train.shape)
print('y_train : ', y_train.shape)
print('X_test : ', X_test.shape)
print('y_test : ', y_test.shape)

X_train :  (126, 7)
y_train :  (126,)
X_test :  (32, 7)
y_test :  (32,)


In [0]:
def err_plot(error_d):
  
  fig = go.Figure()

  for i in error_d.columns[1:]:
    fig.add_trace(go.Scatter(x = error_d.iloc[:,0],
                             y = error_d[i],
                             name = i.split('_')[2],
                             line = dict(width=3)))
  
  fig.update_layout(
      autosize=False,
      width=1100,
      height=650,
      title='Alphs Vs Metric',
      title_x = 0.5,
      xaxis_title = 'Alpha',
      yaxis_title = 'Computed Score/Error'
  )

  fig.show()

In [0]:
def train_test_plot(tr_ts_er):
  fig = go.Figure(data=[
                      go.Bar(x=tr_ts_er.iloc[:,0], y=tr_ts_er.iloc[:,1], name = tr_ts_er.columns[1]),
                      go.Bar(x=tr_ts_er.iloc[:,0], y=tr_ts_er.iloc[:,2], name = tr_ts_er.columns[2])
  ])

  fig.update_layout(barmode='group',
                    autosize=False,
                    width =800,
                    height=650,
                    title='Train Vs Test Error',
                    title_x = 0.5,
                    xaxis_title = 'Alpha',
                    yaxis_title = 'Computed Score/Error')
  fig.show()

In [0]:
def training_model(alpha_r2, alpha_mse, alpha_rmse, r2_vals, mse, rmse, reg, loss, X_train, y_train, X_test, y_test):

  #training the model with found best C and testing on x_test data
  #alpha wrt r2
  model_sgd_r2 = SGDRegressor(alpha = alpha_r2, penalty=reg, loss = loss, random_state=42)
  model_sgd_r2.fit(X_train,y_train)
  y_pred_r2 = model_sgd_r2.predict(X_test)
  
  predic_r2 = X_test.copy()
  predic_r2['Pred'] = y_pred_r2

  sgd_r2_test = r2_score(y_test, y_pred_r2)

  print(f'r2_score at alpha : {alpha_r2} on CV/train data is {r2_vals[alpha_r2]}')
  print(f'r2_score at alpha : {alpha_r2} on test data is {sgd_r2_test}')
  print(' ')

  tr_ts_vals= list([list([str(alpha_r2)+'(r2)',r2_vals[alpha_r2], sgd_r2_test])])

  #alpha wrt mse
  model_lgr_mse = SGDRegressor(alpha = alpha_mse, penalty=reg, loss = loss, random_state=42)
  model_lgr_mse.fit(X_train,y_train)
  y_pred_mse = model_lgr_mse.predict(X_test)

  predic_mse = X_test.copy()
  predic_mse['Pred'] = y_pred_mse

  sgd_mse_test = mean_squared_error(y_test, y_pred_mse)

  print(f'MSE at alpha : {alpha_mse} on CV/train data is {mse[alpha_mse]}')
  print(f'MSE at alpha : {alpha_mse} on test data is {sgd_mse_test}')
  print(' ')

  tr_ts_vals.append(list([str(alpha_mse)+'(MSE)',mse[alpha_mse], sgd_mse_test]))

  #alpha wrt rmse
  model_sgd_rmse = SGDRegressor(alpha = alpha_rmse, penalty=reg, loss = loss, random_state=42)
  model_sgd_rmse.fit(X_train,y_train)
  y_pred_rmse = model_sgd_rmse.predict(X_test)
  
  predic_rmse = X_test.copy()
  predic_rmse['Pred'] = y_pred_rmse

  sgd_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_rmse))

  print(f'rmse at alpha : {alpha_rmse} on CV/train data is {rmse[alpha_rmse]}')
  print(f'rmse at alpha : {alpha_rmse} on test data is {sgd_rmse_test}')
  print(' ')

  tr_ts_vals.append(list([str(alpha_rmse)+'(RMSE)', rmse[alpha_rmse], sgd_rmse_test]))

  tr_ts_er_df = pd.DataFrame(data=tr_ts_vals, columns=['alpha', 'Train', 'Test'])

  train_test_plot(tr_ts_er_df)

  return predic_r2, predic_mse, predic_rmse

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error, make_scorer

def best_alpha_cv(alpha_limit, reg, loss, X_train, y_train, X_test, y_test):

  def sqrot(yt,yp): return np.sqrt(mean_squared_error(yt,yp))

  metrics ={'r2':make_scorer(r2_score), 
            'mse':make_scorer(mean_squared_error, greater_is_better=False),
            'rmse':make_scorer(sqrot, greater_is_better=False)}

  model_sgd = SGDRegressor(penalty = reg, loss = loss, random_state = 42)
  model_gds = RandomizedSearchCV(estimator=model_sgd, param_distributions = alpha_limit, cv=5, scoring=metrics,
                                 refit = False, n_iter=30)
  model_gds.fit(X_train, y_train)

  error_df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in model_gds.cv_results_.items()]))

  r2_vals = dict(zip(error_df.param_alpha, error_df.mean_test_r2))
  mse = dict(zip(error_df.param_alpha, error_df.mean_test_mse))
  rmse = dict(zip(error_df.param_alpha, error_df.mean_test_rmse))

  #finding best alpha wrt each error
  alpha_r2 = max(r2_vals, key=r2_vals.get)
  min_r2 = r2_vals[alpha_r2]

  alpha_mse = min(mse,key=mse.get)
  min_mse = mse[alpha_mse]

  alpha_rmse = min(rmse, key=rmse.get)
  min_rmse = rmse[alpha_rmse]

  #plotting errors vs alpha
  err_plot(error_df[['param_alpha','mean_test_r2','mean_test_mse','mean_test_rmse']])


  #training the model with found best C and testing on x_test data
  print('Training the model with found best alpha')
  predic_r2, predic_mse, predic_rmse = training_model(alpha_r2, alpha_mse, alpha_rmse, r2_vals, mse, rmse, reg, loss, X_train, y_train, X_test, y_test)

  return r2_vals, mse, rmse

In [0]:
from sklearn.utils.fixes import loguniform
from sklearn.model_selection import RandomizedSearchCV

In [0]:
#driver function
alpha_limit = {'alpha': loguniform(1e5, 1e13)}
reg = 'l1'
loss = 'squared_loss'
r2, mse, rmse = best_alpha_cv(alpha_limit, reg, loss, X_train, y_train, X_test, y_test)

Training the model with found best alpha
r2_score at alpha : 12670373513.148096 on CV/train data is -0.04874270630426007
r2_score at alpha : 12670373513.148096 on test data is -0.023779632452066934
 
MSE at alpha : 12670373513.148096 on CV/train data is -0.9907503142179476
MSE at alpha : 12670373513.148096 on test data is 1.0809749118104697
 
rmse at alpha : 12670373513.148096 on CV/train data is -0.9818331781476026
rmse at alpha : 12670373513.148096 on test data is 1.039699433399129
 


In [0]:
[print(loguniform(1e0, 1e3))]

[None]

In [0]:
np.uniform(1,3)

AttributeError: ignored